In [1]:
from core.libs import pd, tqdm
from core.paths import resolve_inventory_paths
from CruisesProcessor.utils.metadata_extractor import extract_metadata_from_excel
from core.db import get_engine

engine = get_engine()

📄 Extrayendo metadatos...
💻 Conectado a la base de datos helloworldtree


In [3]:
# 1. Trae todos los registros con inventory_date = 'pending'
query = """
    SELECT contract_code, rel_path
    FROM masterdatabase.inventory_metrics
    WHERE inventory_date = 'pending'
"""
df_pending = pd.read_sql(query, engine)

if df_pending.empty:
    print("No hay registros pendientes.")
    exit(0)

print(df_pending.head(5))

  contract_code                                           rel_path
0        US0057  USA/2023_ForestInventory/4-Contractor Cruises/...
1        US0040  USA/2023_ForestInventory/4-Contractor Cruises/...
2        US0047  USA/2023_ForestInventory/4-Contractor Cruises/...
3        US0056  USA/2023_ForestInventory/4-Contractor Cruises/...
4        US0067  USA/2023_ForestInventory/4-Contractor Cruises/...


In [9]:
from sqlalchemy import text

# 2. Filtra rel_paths válidos (no None, no vacíos)
df_valid = df_pending[df_pending['rel_path'].notna() & (df_pending['rel_path'] != '')]
paths_absolutas = resolve_inventory_paths(df_valid['rel_path'].tolist())

errores = []
actualizados = 0
sin_fecha = 0

for idx, (contract_code, rel_path_abs) in tqdm(
        enumerate(zip(df_valid['contract_code'], paths_absolutas)), total=len(paths_absolutas)):
    try:
        meta = extract_metadata_from_excel(rel_path_abs)
        cruise_date = meta.get("cruise_date")
        if cruise_date and cruise_date != '' and cruise_date != "pending":
            # UPDATE con fecha real
            q = text("""
                UPDATE masterdatabase.inventory_metrics
                SET inventory_date = :cruise_date
                WHERE contract_code = :contract_code
            """)
            with engine.begin() as conn:
                conn.execute(q, {"cruise_date": cruise_date, "contract_code": contract_code})
            print(f"✅ Actualizado {contract_code}: {cruise_date}")
            actualizados += 1
        else:
            # UPDATE con "Excel without cruise date"
            q = text("""
                UPDATE masterdatabase.inventory_metrics
                SET inventory_date = :cruise_date
                WHERE contract_code = :contract_code
            """)
            with engine.begin() as conn:
                conn.execute(q, {"cruise_date": "Excel without cruise date", "contract_code": contract_code})
            print(f"⚠️ Sin fecha para {contract_code}: marcado como 'Excel without cruise date'")
            sin_fecha += 1
    except Exception as e:
        print(f"❌ Error en {contract_code} ({rel_path_abs}): {e}")
        errores.append((contract_code, str(e)))

print(f"\nListo. Total actualizados: {actualizados} / {len(df_valid)}")
print(f"Total marcados como 'Excel without cruise date': {sin_fecha}")
if errores:
    print("\nErrores:")
    for c, err in errores:
        print(f"{c}: {err}")


  0%|          | 0/104 [00:00<?, ?it/s]C:\Users\HeyCe\.conda\envs\cartografunk\Lib\site-packages\openpyxl\reader\excel.py:237: UserWarning: Data Validation extension is not supported and will be removed
  ws_parser.bind_all()
C:\Users\HeyCe\.conda\envs\cartografunk\Lib\site-packages\openpyxl\reader\excel.py:237: UserWarning: Conditional Formatting extension is not supported and will be removed
  ws_parser.bind_all()
  1%|          | 1/104 [00:00<00:40,  2.53it/s]

✅ Actualizado US0057: 2024-04-11 00:00:00


  2%|▏         | 2/104 [00:00<00:44,  2.30it/s]

✅ Actualizado US0040: 2024-04-02 00:00:00


  3%|▎         | 3/104 [00:01<00:37,  2.71it/s]

...

✅ Actualizado US0073: 2024-04-12 00:00:00


 99%|█████████▉| 103/104 [02:24<00:00,  2.06it/s]

✅ Actualizado US0083: 2024-04-12 00:00:00


100%|██████████| 104/104 [02:55<00:00,  1.68s/it]

⚠️ Sin fecha para CR0028: marcado como 'Excel without cruise date'

Listo. Total actualizados: 94 / 104
Total marcados como 'Excel without cruise date': 10
